In [ ]:
import mysql.connector
import os

# MySQL connection details
host = 'localhost'  # MySQL server host (e.g., localhost or IP address)
user = 'root'       # MySQL username
password = 'password'  # MySQL password

# Directory to save the SQL files
output_dir = 'D:/mysql_databases_sql/'

# Create output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Connect to MySQL
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password
    )
    cursor = conn.cursor()

    # Get all databases
    cursor.execute("SHOW DATABASES")
    databases = cursor.fetchall()

    for db in databases:
        db_name = db[0]
        print(f"Processing database: {db_name}")

        # Create a file for the database
        db_file = os.path.join(output_dir, f"{db_name}.sql")

        # Open the file to write the SQL
        with open(db_file, 'w') as file:
            # Write CREATE DATABASE statement
            file.write(f"CREATE DATABASE IF NOT EXISTS `{db_name}`;\n")
            file.write(f"USE `{db_name}`;\n\n")

            # Fetch tables and generate CREATE TABLE statements
            cursor.execute(f"SHOW TABLES FROM `{db_name}`")
            tables = cursor.fetchall()

            for table in tables:
                table_name = table[0]
                cursor.execute(f"SHOW CREATE TABLE `{db_name}`.`{table_name}`")
                create_table = cursor.fetchone()[1]
                file.write(f"\n-- Table: {table_name}\n")
                file.write(f"{create_table};\n")

            # Fetch triggers and generate CREATE TRIGGER statements
            cursor.execute(f"SHOW TRIGGERS FROM `{db_name}`")
            triggers = cursor.fetchall()

            for trigger in triggers:
                trigger_name = trigger[0]
                trigger_event = trigger[1]
                trigger_timing = trigger[2]
                trigger_table = trigger[3]

                # Fetch the full CREATE TRIGGER statement for each trigger
                cursor.execute(f"SHOW CREATE TRIGGER `{db_name}`.`{trigger_name}`")
                create_trigger = cursor.fetchone()[2]  # This contains the trigger's full definition

                # Properly write the CREATE TRIGGER statement
                file.write(f"\n-- Trigger: {trigger_name} ({trigger_timing} {trigger_event} ON {trigger_table})\n")
                file.write(f"{create_trigger};\n")

            # Fetch procedures and generate CREATE PROCEDURE statements
            cursor.execute(f"SHOW PROCEDURE STATUS WHERE Db = '{db_name}'")
            procedures = cursor.fetchall()

            for proc in procedures:
                proc_name = proc[1]
                cursor.execute(f"SHOW CREATE PROCEDURE `{db_name}`.`{proc_name}`")
                create_proc = cursor.fetchone()[2]
                file.write(f"\n-- Procedure: {proc_name}\n")
                file.write(f"{create_proc};\n")

            # Fetch functions and generate CREATE FUNCTION statements
            cursor.execute(f"SHOW FUNCTION STATUS WHERE Db = '{db_name}'")
            functions = cursor.fetchall()

            for func in functions:
                func_name = func[1]
                cursor.execute(f"SHOW CREATE FUNCTION `{db_name}`.`{func_name}`")
                create_func = cursor.fetchone()[2]
                file.write(f"\n-- Function: {func_name}\n")
                file.write(f"{create_func};\n")

        print(f"SQL file for {db_name} saved at {db_file}")

except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    cursor.close()
    conn.close()


In [ ]:
import mysql.connector
import os

# MySQL connection details
host = 'localhost'  # MySQL server host (e.g., localhost or IP address)
user = 'root'       # MySQL username
password = 'password'  # MySQL password

# Directory where the SQL backup files are stored
backup_dir = '/media/rajesh/Rajesh/mysql_databases_sql'
# backup_dir = 'D:/mysql_databases_sql/'

# Connect to MySQL
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password
    )
    cursor = conn.cursor()

    # Get a list of all .sql files in the backup directory
    backup_files = [f for f in os.listdir(backup_dir) if f.endswith('.sql')]

    for backup_file in backup_files:
        db_name = backup_file.replace('.sql', '')  # Extract the database name from the file name
        print(f"Restoring database: {db_name} from {backup_file}")

        # Read the backup SQL file
        with open(os.path.join(backup_dir, backup_file), 'r') as file:
            sql_commands = file.read()

        # Start by creating the database if it doesn't exist
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{db_name}`;")
        cursor.execute(f"USE `{db_name}`;")

        # Split the SQL commands by delimiter (handles multiple statements in the file)
        # The DELIMITER keyword is used to define the multi-line SQL statements (e.g., for triggers)
        sql_commands = sql_commands.split(';')

        # For each command, check if it's a trigger, procedure, or function and handle accordingly
        current_command = ""
        inside_delimiter = False
        for command in sql_commands:
            command = command.strip()

            if command.lower().startswith("delimiter"):
                # Skip the delimiter lines as they are for special handling
                inside_delimiter = True
                continue
            
            if inside_delimiter:
                # Handle multi-line objects (triggers, procedures, functions)
                current_command += command + ";"
                if command.endswith("END;"):  # Trigger, procedure or function might end with "END;"
                    try:
                        cursor.execute(current_command)
                        print(f"Executed: {current_command[:50]}...")  # Print a snippet of the command
                    except mysql.connector.Error as err:
                        print(f"Error executing: {current_command[:50]}... Error: {err}")
                    current_command = ""
                    inside_delimiter = False
            else:
                if command:  # Ignore empty commands
                    try:
                        cursor.execute(command + ";")
                        print(f"Executed: {command[:50]}...")  # Print a snippet of the command
                    except mysql.connector.Error as err:
                        print(f"Error executing: {command[:50]}... Error: {err}")

        print(f"Database {db_name} restored successfully!")

except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    cursor.close()
    conn.close()


In [ ]:
import subprocess

# MySQL credentials
user = 'root'
password = 'password'
database = 'customersdb'

# Backup file location
backup_path = 'D:\\customersdb_backup.sql'

# Build the command
command = [
    r'C:\Program Files\MySQL\MySQL Server 8.0\bin\mysqldump',
    f'-u{user}',
    f'-p{password}',
    '--triggers',
    database
]

# Open file to write the backup
with open(backup_path, 'w') as backup_file:
    try:
        subprocess.run(command, stdout=backup_file, check=True)
        print(f'Backup completed: {backup_path}')
    except subprocess.CalledProcessError as e:
        print('Backup failed:', e)


In [ ]:
def t():
    import mysql.connector

    # MySQL connection settings
    config = {
        'host': 'localhost',
        'user': 'root',
        'password': 'password',
        'database': 'customersdb'
    }

    try:
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor()

        # Disable foreign key checks
        cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")

        # Get list of all tables
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = %s AND table_type = 'BASE TABLE';", (config['database'],))
        tables = cursor.fetchall()

        # Truncate each table
        for (table,) in tables:
            print(f"Truncating table: {table}")
            cursor.execute(f"TRUNCATE TABLE `{table}`;")

        # Re-enable foreign key checks
        cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

        conn.commit()
        print("truncated successfully.")

    except mysql.connector.Error as err:
        print(f"error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()


In [ ]:
import mysql.connector

# Connect to the database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='password',
    database='customersdb'
)

cursor = conn.cursor()

# Get list of tables
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

# Loop through and get CREATE TABLE statement for each
for (table_name,) in tables:
    cursor.execute(f"SHOW CREATE TABLE `{table_name}`")
    result = cursor.fetchone()
    print(f"\n--- {table_name} ---")
    print(result[1])  # The CREATE TABLE statement

cursor.close()
conn.close()
